In [1]:
import time
import json
import sys
import os
import glob
import re
from collections import deque

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [2]:
fxc = FuncXClient()
# fxs = FuncXSerializer() # (de)serialize the funcx codes

In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    from subprocess import PIPE
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #recon. script parameters
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_ngpu = data['rec_ngpu']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --folder='{ofpath}'"

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

66113be9-af59-440b-bf3a-b850f4818dcd


In [4]:
def get_folder_paths(path):
    import glob
    import re
    
    #return f"{path}/teko007"
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))

def get_file_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

6a2b9010-85a8-4eaa-a50f-235b4415ce53
803d5584-57a9-4370-9582-354399dcc233


In [5]:
def copy_file(params):
    import os
    import shutil
    
    ifpath = params['ifpath']
    ofpath = params['ofpath']
    
    #shutil.copyfile(ifpath, ofpath)
    shutil.copytree(ifpath, ofpath)

def my_hello():
    return "Welcome to the machine"

func_cp_uuid = fxc.register_function(copy_file)
func_hl_uuid = fxc.register_function(my_hello)
print(func_cp_uuid)

5d61d781-c39b-4d88-b67b-4130f3ec333e


In [6]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 1800,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 3600,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 1800,
      "End": True
    },
  }
}

In [7]:
prisma_fx_endpoint = '4bf59543-3398-42d2-9416-c628e9f5635f'

src_wf_root_path = '/prisma-data1/bicer/workflow' #'/gdata/RAVEN/bicer/2020-1/comm_33IDD/globus_automate'
src_input_folder_prefix = "input"
src_output_folder_prefix = "output"

dest_wf_root_path = '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate'
dest_input_folder_prefix = "input"
dest_output_folder_prefix = "output"


#src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*.h5"
src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
rid = fxc.run(src_input_folder_paths_regex, 
              endpoint_id=prisma_fx_endpoint, 
              function_id=fx_func_get_folder_paths_uuid)
src_input_folder_paths = fxc.get_result(rid)

src_output_folder_paths = []
dest_output_folder_paths = []
dest_input_folder_paths = []
for src_input_folder_path in src_input_folder_paths:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
    src_output_folder_paths.append(src_output_folder_path)
    dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
    dest_input_folder_paths.append(dest_input_folder_path)
    dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
    dest_output_folder_paths.append(dest_output_folder_path)
    
    

# src_input_folder_paths: diffraction patterh files to be processed @ APS
# src_output_folder_paths: folders for reconstrcuted images after processing @ APS
# dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
# dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

for (src_input_folder_path, src_output_folder_path, dest_input_folder_path, dest_output_folder_path ) in zip(src_input_folder_paths, src_output_folder_paths, dest_input_folder_paths, dest_output_folder_paths):
    print(f"Source input folder: {src_input_folder_path}; Source output folder: {src_output_folder_path}")
    print(f"Dest. input folder: {dest_input_folder_path}; Dest. output folder: {dest_output_folder_path}")
    print()

Source input folder: /prisma-data1/bicer/workflow/input/scan192; Source output folder: /prisma-data1/bicer/workflow/output/192
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/192; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/192

Source input folder: /prisma-data1/bicer/workflow/input/scan193; Source output folder: /prisma-data1/bicer/workflow/output/193
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/193; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/193

Source input folder: /prisma-data1/bicer/workflow/input/scan194; Source output folder: /prisma-data1/bicer/workflow/output/194
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/194; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/194

Source input folder: /prisma-data1/bicer/workflow/input/scan195; Source output folder

In [8]:
theta_fx_endpoint = 'f765db7a-038c-47ea-9176-d81de31c054f' #'7f42390d-849a-42a7-905c-db6b22af28f7' # #FuncX endpoint

#src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' #'hostel' #aps/workstation
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN # 'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #

# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/mpi_tike-recon.py'
python_path = "/home/bicer/miniconda3/envs/ptycho/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 8
rec_upd_pos = False
rec_niter = 10
rec_output_freq = 5
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 1
rec_use_mpi = False

flow_inputs = []

for (src_input_folder_path, src_output_folder_path, 
     dest_input_folder_path, dest_output_folder_path ) in zip(
    src_input_folder_paths, src_output_folder_paths, 
    dest_input_folder_paths, dest_output_folder_paths):
    
    flow_input = {
        "input": {
            "source_endpoint": f"{src_endpoint}",
            "source_path": f"{src_input_folder_path}",
            "dest_endpoint": dest_endpoint,
            "dest_path": f"{dest_input_folder_path}",

            "result_path": f"{dest_output_folder_path}",#f"{dest_resultpath}/out_{src_filename}",
            "source_result_path": f"{src_output_folder_path}", #/out_{src_filename}",
            "fx_ep": f"{theta_fx_endpoint}",
            "fx_id": f"{func_cp_uuid}", #f"{func_hl_uuid}" #
            #"fx_id": f"{func_ptycho_uuid}",
            #"params": {'ifpath': f"{dest_input_folder_path}",
            #           'ofpath': f"{dest_output_folder_path}/",
            #           'script_path': script_path,
            #           'python_path': python_path,
            #           'rec_alg': rec_alg,
            #           'rec_nmodes': rec_nmodes,
            #           'rec_upd_pos': rec_upd_pos,
            #           'rec_niter': rec_niter,
            #           'rec_output_freq': rec_output_freq,
            #           'rec_recover_psi': rec_recover_psi,
            #           'rec_recover_probe': rec_recover_probe,
            #           'rec_recover_positions': rec_recover_positions,
            #           'rec_model': rec_model,
            #           'rec_ngpu': rec_ngpu,
            #           'rec_use_mpi': rec_use_mpi}
            "params": {"ifpath": f"{dest_input_folder_path}",
                       "ofpath": f"{dest_output_folder_path}"}
        }
    }
    flow_inputs.append(flow_input)

#print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
#print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
#print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

In [9]:
print(len(flow_inputs))

168


In [10]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

print(flow)
#print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/e66c2ee5-8e7f-4296-997b-b791131052df', 'administered_by': [], 'api_version': '1.0', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input.params'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 3600}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destination_endpoint_id.$': '$.input.dest_endpoint', 'sou

In [11]:
MAX_QSIZE =128
q0 = deque()
q1 = deque()

for i in range(len(flow_inputs)):
    if i<MAX_QSIZE:
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i])
        q1.append(flow_action)
        print(f"Flow {i} initiated and added to q1: {flow_action['action_id']}")
    else: 
        q0.append(flow_inputs[i])
        print(f"Flow input {i} added to q0")

i=-1
while len(q1)>0:
    #time.sleep(0.5)
    i = (i+1)%len(q1)
    flow = flows_client.flow_action_status(flow_id, flow_scope, q1[i]['action_id'])
    print(f"len(q0)={len(q0)}; len(q1)={len(q1)}; i={i}")
    print(f"Flow {i} status: {q1[i]['action_id']}: {flow['status']}")
    if flow['status'] == 'SUCCEEDED':
        del q1[i]
        if len(q0)>0:
            flow_input = q0.popleft()
            flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
            q1.append(flow_action)
            print(f"New flow initiated and added to the q1: {flow_action['action_id']}")
            print(f"Copy from {flow_input['input']['params']['ifpath']} to {flow_input['input']['params']['ofpath']}")


Starting login with Globus Auth, press ^C to cancel.
Flow 0 initiated and added to q1: c7d93139-f573-448a-b752-085c690f1918
Flow 1 initiated and added to q1: 81fe19f0-08f2-422a-91d5-aaa86462cf99
Flow 2 initiated and added to q1: e4e2db74-ba5f-4905-b5f6-b0026411fe5d
Flow 3 initiated and added to q1: 89e0a874-fd56-41d5-b5c3-5ef76c6b70b7
Flow 4 initiated and added to q1: 58a61961-251a-4d38-878a-6b2254e2abdf
Flow 5 initiated and added to q1: 06493b02-2ced-4640-93e6-b478b145276f
Flow 6 initiated and added to q1: 1c067620-9c05-4edb-a12c-1e35c33b9f94
Flow 7 initiated and added to q1: 29170eed-3b35-4bd3-90e3-789732b0e9da
Flow 8 initiated and added to q1: 257b3057-3c9f-46c2-932a-074cc68fc7bc
Flow 9 initiated and added to q1: e0dc02ec-242c-4890-8550-83b141eb4c49
Flow 10 initiated and added to q1: 8ad6ba1a-756f-4d40-8537-203fcf4fda6d
Flow 11 initiated and added to q1: 4ed2333b-8aa1-4dfe-9303-6f2b3201f65a
Flow 12 initiated and added to q1: daba3748-2672-4091-9d9d-7f12023a0bca
Flow 13 initiated and

Flow 114 initiated and added to q1: 8b3eb625-a741-48a7-a104-bfdffb292b4f
Flow 115 initiated and added to q1: a78dcaf7-49c0-4f20-8c65-71caf0c785ae
Flow 116 initiated and added to q1: 5bfb28c4-9ebb-46f6-921d-ecc70bedb9a0
Flow 117 initiated and added to q1: 118ff4a9-aee1-4d19-9b80-35b77d65809c
Flow 118 initiated and added to q1: 9c59713e-b100-400a-ab6e-58a47296dd12
Flow 119 initiated and added to q1: d9d33256-f3da-486f-b4fc-570337b250ad
Flow 120 initiated and added to q1: 91e31d7b-576d-4663-b42b-3d2c25695196
Flow 121 initiated and added to q1: e2bf85d5-e4f9-41b3-8069-b2af7b29c845
Flow 122 initiated and added to q1: 7d231456-7cdb-4173-8d0d-e196a689a762
Flow 123 initiated and added to q1: c5ac6131-2124-45cd-9e3d-a71310ce88a3
Flow 124 initiated and added to q1: 2b19c83c-a8e2-4c32-a568-37c8446646ea
Flow 125 initiated and added to q1: 333892dd-d4df-4f51-918f-8a13f6e3d3ae
Flow 126 initiated and added to q1: 32b4c3db-fef6-4b55-a660-fecd55f12747
Flow 127 initiated and added to q1: 4df2b814-340a-4

New flow initiated and added to the q1: 3142d816-8cf7-450e-a362-006a5b72d84f
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/340 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/340
len(q0)=20; len(q1)=128; i=20
Flow 20 status: 81b88613-11e5-47d2-90fd-a2b152ec0c2e: SUCCEEDED
New flow initiated and added to the q1: ec54c917-cfe8-422e-81b1-1eabb4f9d562
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/341 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/341
len(q0)=19; len(q1)=128; i=21
Flow 21 status: bf28d20a-0591-4a08-b00a-b827d84b2491: SUCCEEDED
New flow initiated and added to the q1: 2af04b18-0c27-4fef-a451-70a4194cf7bc
Copy from /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/342 to /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/342
len(q0)=18; len(q1)=128; i=22
Flow 22 status: 9b1bdfb5-4b34-49f2-a443-199fa9f523e9: SUCCEEDED
New flow initiated and added to the q1: d04

len(q0)=0; len(q1)=110; i=58
Flow 58 status: 5bfb28c4-9ebb-46f6-921d-ecc70bedb9a0: SUCCEEDED
len(q0)=0; len(q1)=109; i=59
Flow 59 status: 9c59713e-b100-400a-ab6e-58a47296dd12: SUCCEEDED
len(q0)=0; len(q1)=108; i=60
Flow 60 status: 91e31d7b-576d-4663-b42b-3d2c25695196: SUCCEEDED
len(q0)=0; len(q1)=107; i=61
Flow 61 status: 7d231456-7cdb-4173-8d0d-e196a689a762: SUCCEEDED
len(q0)=0; len(q1)=106; i=62
Flow 62 status: 2b19c83c-a8e2-4c32-a568-37c8446646ea: SUCCEEDED
len(q0)=0; len(q1)=105; i=63
Flow 63 status: 32b4c3db-fef6-4b55-a660-fecd55f12747: SUCCEEDED
len(q0)=0; len(q1)=104; i=64
Flow 64 status: 6bdb6e0b-a661-489a-acc0-78e1b8b90ede: SUCCEEDED
len(q0)=0; len(q1)=103; i=65
Flow 65 status: c6cff194-d37b-4b4e-8bf9-5bd38c8fa46e: SUCCEEDED
len(q0)=0; len(q1)=102; i=66
Flow 66 status: 0c808394-11dd-4c00-b70b-df877eecdfc4: SUCCEEDED
len(q0)=0; len(q1)=101; i=67
Flow 67 status: 3be91208-7eb0-47bd-85ee-f55496fc4e8f: SUCCEEDED
len(q0)=0; len(q1)=100; i=68
Flow 68 status: 2df2fb94-0b9c-4fd5-b80f-8

len(q0)=0; len(q1)=23; i=20
Flow 20 status: 2014c77a-f572-4b93-ae44-11e00b21632c: SUCCEEDED
len(q0)=0; len(q1)=22; i=21
Flow 21 status: cdc35997-8999-420e-8a3a-aabc5fd3b1d1: SUCCEEDED
len(q0)=0; len(q1)=21; i=1
Flow 1 status: 17266567-c5c0-43e3-b8fb-309578242b6f: SUCCEEDED
len(q0)=0; len(q1)=20; i=2
Flow 2 status: ee43b122-75b7-4fa3-a4af-805b3230bf5e: SUCCEEDED
len(q0)=0; len(q1)=19; i=3
Flow 3 status: b4f2945b-7fca-4a5b-b4ab-350ceaae515f: SUCCEEDED
len(q0)=0; len(q1)=18; i=4
Flow 4 status: e4980640-a04b-4222-aae9-74fff34518e5: SUCCEEDED
len(q0)=0; len(q1)=17; i=5
Flow 5 status: e8bb5f3b-46e6-454d-a80f-6a8f8edd00b5: SUCCEEDED
len(q0)=0; len(q1)=16; i=6
Flow 6 status: 50eb8da0-60ce-4369-8c1e-0859f309bee8: SUCCEEDED
len(q0)=0; len(q1)=15; i=7
Flow 7 status: 0f4bbb2d-5707-44d2-9f44-335e9502f50f: SUCCEEDED
len(q0)=0; len(q1)=14; i=8
Flow 8 status: 333892dd-d4df-4f51-918f-8a13f6e3d3ae: SUCCEEDED
len(q0)=0; len(q1)=13; i=9
Flow 9 status: f1a97d0b-959a-4aee-bd42-d69d76e625e5: SUCCEEDED
len(q0

In [ ]:
print(flow_scope)

In [12]:
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])

In [ ]:
#flows_client.flow_action_cancel(flow_id, flow_scope, '5e948f48-c3c3-4328-8d37-b5d1bc82df4c')

In [ ]:
#for ga_flow_action_id in ga_flow_actions.keys():
#    flows_client.flow_action_cancel(flow_id, flow_scope, ga_flow_action_id)

In [ ]:
#st = flows_client.flow_action_status(flow_id, flow_scope, '08925f04-569f-11e7-bef8-22000b9a448b')
#print(st['status'])